## **Web Scraping de TUCARRO.COM paginas particulares**

### **1. Importamos las respectivas librerias**

In [ ]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import re
import json

### **2. Hacemos el respectivo requerimiento a la pagina**

➡️ Tenemos que hacer la peticion de conexion a la pagina que vamos a scrapear

In [ ]:
html_text = requests.get('https://articulo.tucarro.com.co/MCO-2526957066-mercedes-benz-cla-180-13-amg-line-2022-_JM#position%3D1%26search_layout%3Dgrid%26type%3Ditem%26tracking_id%3Ddd2134b9-b228-49f8-a6c2-33ab624ba167')

➡️ Necesitamos un indicador que nos diga si la conexion fue realmente efectiva, en este caso, **200** es el código que muestra una conexion exitosa.

🗒️ **Observación:** Para una mayor informacion, les dejo el siguiente link 🔗 https://developer.mozilla.org/es/docs/Web/HTTP/Status/200

In [ ]:
status_code = html_text.status_code
print(status_code)

➡️ Ahora, vamos a convertir la url de la pagina en la que estamos situados en un valor de referencia que utilizaremos posteriormente

In [ ]:
conexion_url = html_text.url
print(conexion_url)

### **3. Seleccionamos las grillas especificas de donde vamos a sacar la informacion**

In [ ]:
soup = BeautifulSoup(html_text.text, 'lxml')

➡️ En esta pagina hay bastante informacion, sin embargo, la primera parte que dice *"Caracteristicas del producto"* pareciera estar ya contenida en las tablas que se muestran justo abajo. Por decision, vamos a tomar solo la informacion que se detalla en las tablas.

⚠️ **Alerta:** Esta parte de la pagina pareciera esta protegida para hacer *web scraping*, por lo tanto debemos excabar para ver que informacion si es posible extraer.

#### 3.1 Condiciones de compra

➡️ Exploremos la informacion que se esta extrayendo en el request

In [ ]:
print(soup)

➡️ La unica informacion que pareciera estar disponible tien que ver con *Color*, *Tipo de carroceria*, *Marca*, etc. Vamos a extraer esa informacion. El formato esta en formato JSON asi que vamos a extraer la informacion teniendo este hecho presente.

In [ ]:
# Buscamos en elemento script dentro del bloque del html
script = soup.find('script', type='application/ld+json')

# Extraer el contenido del script como texto. Strip se encargara de eliminar los espacios vacios al principio y al final de la cadena
script_content = script.text.strip()

# Convertir el contenido del script a un objeto JSON
data = json.loads(script_content)
data

➡️ Ahora vamos a crear un dataframe a partir de la información que conseguimos.

🗒️ **Observación:** Vamos a crear un dataframe donde una columna contenga el nombre de la característica y otra con los valores asociados.

In [ ]:
features_ONLY_offers = list(data['offers'].keys())
features_ONLY_offers

In [ ]:
features_NO_offers = list(data.keys())
features_NO_offers = [item for item in features_NO_offers if item != 'offers']
features_NO_offers

In [ ]:
features = features_ONLY_offers + features_NO_offers
features

➡️ Ahora deberíamos hacer lo mismo con los valores.

In [ ]:
values_ONLY_offers = list(data['offers'].values())
values_NO_offers = [value for key, value in data.items() if key != 'offers']
values = values_ONLY_offers + values_NO_offers
values

➡️ Convertimos en un dicionario las listas previamente definidas. Utilizamos la funcion 📦 *zip* para unir en pares las llaves y los valores. Adicionalmente, creamos un dataframe a partir de este diccionario utilizando el atributo **index=[0]** que nos ayuda a dejar como nombre de columnas las llaves con el valor respectivo.

In [ ]:
data_dict = {str(feature): value for feature, value in zip(features, values)}
df = pd.DataFrame(data_dict, index=[0])

🗒️ **Observación:** No olvidemos agregar el registro ('timestamp') de la operación y la página de origen

In [ ]:
df['Registro'] = pd.Timestamp.now()
df['Origen_individual'] = conexion_url
df